In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy
from tensorflow.keras.callbacks import EarlyStopping ,ModelCheckpoint ,TensorBoard
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_data_dir ="C:\\Users\\Shehab\\Desktop\\Finish\\Gender\\Gender\\train"
train_datagen = ImageDataGenerator(rescale=1./255 , validation_split=0.25)
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(128, 128 ),
                                                    batch_size=32,
                                                    class_mode='binary',
                                                   subset='training' )

validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='validation')

In [ ]:
class_labels = list(train_generator.class_indices.keys())
class_counts = train_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class - Train Dataset')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:
class_labels = list(validation_generator.class_indices.keys())
class_counts = validation_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:

batch_size=32
num_of_epoches=50

input_shape = (128, 128, 3)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=input_shape),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath="CheckPoint.h5",
                         monitor='val_accuracy',
                         save_best_only=True,
                         save_weights_only=False,
                         verbose=1
                        )
tensorboard = TensorBoard(log_dir="C:\\Users\\Shehab\\Desktop\\Gender")


    


In [ ]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config) 

with tf.device('/GPU:0'):
    history = model.fit(train_generator,
                        steps_per_epoch=train_generator.n // batch_size,
                        epochs=num_of_epoches,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n // batch_size ,
                       verbose=1 , callbacks=[early_stopping ,checkpoint ,tensorboard])

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

plt.savefig('final_cnn_loss_accuracy.png', bbox_inches='tight');

In [ ]:
test_data_dir ="C:\\Users\\Shehab\\Desktop\\Gender\\Gender\\test"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        shuffle=False)


In [ ]:
model.evaluate(test_generator)

In [ ]:
predictions = model.predict(test_generator, steps=test_generator.samples // batch_size + 1)
true_classes = test_generator.classes


In [ ]:
predicted_classes = []
for pred in predictions:
    if pred[0] >= 0.5:
        predicted_classes.append(1)
    else:
        predicted_classes.append(0)


In [ ]:
cm = confusion_matrix(true_classes, predicted_classes)


In [ ]:
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
target_names = ['Female', 'Male']
fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(cm_norm, cmap=plt.cm.Blues, alpha=0.3)
for i in range(cm_norm.shape[0]):
    for j in range(cm_norm.shape[1]):
        ax.text(x=j, y=i, s=format(cm_norm[i, j], '.2f'),
                ha='center', va='center', color='black', fontsize=14)
plt.xticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.yticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.xlabel('Predicted label', fontsize=14)
plt.ylabel('True label', fontsize=14)
plt.title('Confusion matrix', fontsize=16)
plt.show()